In [ ]:
!pip install datasets
!pip install -U bitsandbytes
!pip install peft
import os
import json
import torch
import numpy as np
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

torch.manual_seed(42)
np.random.seed(42)

emoji_math_data = [
    {
        "problem": "🍎 + 🍎 + 🍎 = 12",
        "solution": "To solve this problem, I need to find what value 🍎 represents. Since 🍎 + 🍎 + 🍎 = 12, and there are three identical emoji, each 🍎 must equal 4. So 🍎 = 4."
    },
    {
        "problem": "🍌 + 🍌 = 🍊 + 4",
        "solution": "This is an equation with two unknowns. If 🍌 + 🍌 = 🍊 + 4, then 2🍌 = 🍊 + 4. I need additional information to solve for both values."
    },
    {
        "problem": "2 × 🍕 = 16",
        "solution": "To solve this problem, I need to find what value 🍕 represents. Since 2 × 🍕 = 16, I can divide both sides by 2: 🍕 = 16 ÷ 2 = 8. So 🍕 = 8."
    },
    {
        "problem": "🐱 + 🐱 + 🐱 = 15",
        "solution": "To solve this problem, I need to find what value 🐱 represents. Since 🐱 + 🐱 + 🐱 = 15, and there are three identical emoji, each 🐱 must equal 5. So 🐱 = 5."
    },
    {
        "problem": "🚗 - 🚲 = 10",
        "solution": "This is an equation with two unknowns. If 🚗 - 🚲 = 10, then 🚗 = 10 + 🚲. I need additional information to solve for both values."
    },
    {
        "problem": "🌮 × 4 = 20",
        "solution": "To solve this problem, I need to find what value 🌮 represents. Since 🌮 × 4 = 20, I can divide both sides by 4: 🌮 = 20 ÷ 4 = 5. So 🌮 = 5."
    },
    {
        "problem": "🎸 ÷ 3 = 4",
        "solution": "To solve this problem, I need to find what value 🎸 represents. Since 🎸 ÷ 3 = 4, I can multiply both sides by 3: 🎸 = 4 × 3 = 12. So 🎸 = 12."
    },
    {
        "problem": "🏠 + 🏠 + 🏠 + 🏠 = 40",
        "solution": "To solve this problem, I need to find what value 🏠 represents. Since 🏠 + 🏠 + 🏠 + 🏠 = 40, and there are four identical emoji, each 🏠 must equal 10. So 🏠 = 10."
    },
    {
        "problem": "3 × 🌟 = 27",
        "solution": "To solve this problem, I need to find what value 🌟 represents. Since 3 × 🌟 = 27, I can divide both sides by 3: 🌟 = 27 ÷ 3 = 9. So 🌟 = 9."
    },
    {
        "problem": "🍦 + 🍦 + 🍦 = 🍰 + 🍰",
        "solution": "This is an equation with two unknowns. If 🍦 + 🍦 + 🍦 = 🍰 + 🍰, then 3🍦 = 2🍰, or 🍦 = (2/3)🍰. I need additional information to solve for both values."
    },
    {
        "problem": "🎈 × 3 = 🎁 + 5",
        "solution": "This is an equation with two unknowns. If 🎈 × 3 = 🎁 + 5, then 3🎈 = 🎁 + 5, or 🎈 = (🎁 + 5)/3. I need additional information to solve for both values."
    },
    {
        "problem": "🐶 + 🐶 = 14",
        "solution": "To solve this problem, I need to find what value 🐶 represents. Since 🐶 + 🐶 = 14, and there are two identical emoji, each 🐶 must equal 7. So 🐶 = 7."
    },
    {
        "problem": "🍕 ÷ 2 = 4",
        "solution": "To solve this problem, I need to find what value 🍕 represents. Since 🍕 ÷ 2 = 4, I can multiply both sides by 2: 🍕 = 4 × 2 = 8. So 🍕 = 8."
    },
    {
        "problem": "🚲 × 🚲 = 16",
        "solution": "To solve this problem, I need to find what value 🚲 represents. Since 🚲 × 🚲 = 16, I need to find the square root of 16: √16 = 4. So 🚲 = 4."
    },
    {
        "problem": "5 × 🎭 = 35",
        "solution": "To solve this problem, I need to find what value 🎭 represents. Since 5 × 🎭 = 35, I can divide both sides by 5: 🎭 = 35 ÷ 5 = 7. So 🎭 = 7."
    },
    {
        "problem": "🌈 + 🌈 + 🌈 = 24",
        "solution": "To solve this problem, I need to find what value 🌈 represents. Since 🌈 + 🌈 + 🌈 = 24, and there are three identical emoji, each 🌈 must equal 8. So 🌈 = 8."
    },
    {
        "problem": "🍎 + 🍊 = 11, 🍎 - 🍊 = 3",
        "solution": "I have two equations: 🍎 + 🍊 = 11 and 🍎 - 🍊 = 3. I can solve for both values. From the second equation: 🍎 = 🍊 + 3. Substituting into the first equation: (🍊 + 3) + 🍊 = 11, so 2🍊 + 3 = 11, 2🍊 = 8, 🍊 = 4. Then 🍎 = 🍊 + 3 = 4 + 3 = 7. So 🍎 = 7 and 🍊 = 4."
    },
    {
        "problem": "🚗 + 🚗 + 🚗 + 🚗 = 20",
        "solution": "To solve this problem, I need to find what value 🚗 represents. Since 🚗 + 🚗 + 🚗 + 🚗 = 20, and there are four identical emoji, each 🚗 must equal 5. So 🚗 = 5."
    },
    {
        "problem": "🎩 × 🎩 = 25",
        "solution": "To solve this problem, I need to find what value 🎩 represents. Since 🎩 × 🎩 = 25, I need to find the square root of 25: √25 = 5. So 🎩 = 5."
    },
    {
        "problem": "🌺 ÷ 2 = 6",
        "solution": "To solve this problem, I need to find what value 🌺 represents. Since 🌺 ÷ 2 = 6, I can multiply both sides by 2: 🌺 = 6 × 2 = 12. So 🌺 = 12."
    },
    {
        "problem": "🍌 + 🍌 = 🎁 + 6, 🍌 - 🎁 = 3",
        "solution": "I have two equations: 🍌 + 🍌 = 🎁 + 6 and 🍌 - 🎁 = 3. From the second equation: 🍌 = 🎁 + 3. Substituting into the first equation: (🎁 + 3) + (🎁 + 3) = 🎁 + 6, so 2🎁 + 6 = 🎁 + 6, 🎁 = 0. Then 🍌 = 🎁 + 3 = 0 + 3 = 3. So 🍌 = 3 and 🎁 = 0."
    },
    {
        "problem": "🔑 + 🔑 + 🔑 = 18",
        "solution": "To solve this problem, I need to find what value 🔑 represents. Since 🔑 + 🔑 + 🔑 = 18, and there are three identical emoji, each 🔑 must equal 6. So 🔑 = 6."
    },
    {
        "problem": "🌮 × 5 = 30",
        "solution": "To solve this problem, I need to find what value 🌮 represents. Since 🌮 × 5 = 30, I can divide both sides by 5: 🌮 = 30 ÷ 5 = 6. So 🌮 = 6."
    },
    {
        "problem": "🎂 ÷ 3 = 5",
        "solution": "To solve this problem, I need to find what value 🎂 represents. Since 🎂 ÷ 3 = 5, I can multiply both sides by 3: 🎂 = 5 × 3 = 15. So 🎂 = 15."
    },
    {
        "problem": "🍓 + 🍓 = 🍍 + 2, 🍍 - 🍓 = 1",
        "solution": "I have two equations: 🍓 + 🍓 = 🍍 + 2 and 🍍 - 🍓 = 1. From the second equation: 🍍 = 🍓 + 1. Substituting into the first equation: 🍓 + 🍓 = (🍓 + 1) + 2, so 2🍓 = 🍓 + 3, 🍓 = 3. Then 🍍 = 🍓 + 1 = 3 + 1 = 4. So 🍓 = 3 and 🍍 = 4."
    },
    {
        "problem": "3 × 🐘 = 36",
        "solution": "To solve this problem, I need to find what value 🐘 represents. Since 3 × 🐘 = 36, I can divide both sides by 3: 🐘 = 36 ÷ 3 = 12. So 🐘 = 12."
    },
    {
        "problem": "🍒 + 🍒 + 🍒 + 🍒 = 16",
        "solution": "To solve this problem, I need to find what value 🍒 represents. Since 🍒 + 🍒 + 🍒 + 🍒 = 16, and there are four identical emoji, each 🍒 must equal 4. So 🍒 = 4."
    },
    {
        "problem": "🎮 × 🎮 = 36",
        "solution": "To solve this problem, I need to find what value 🎮 represents. Since 🎮 × 🎮 = 36, I need to find the square root of 36: √36 = 6. So 🎮 = 6."
    },
    {
        "problem": "🚀 ÷ 4 = 5",
        "solution": "To solve this problem, I need to find what value 🚀 represents. Since 🚀 ÷ 4 = 5, I can multiply both sides by 4: 🚀 = 5 × 4 = 20. So 🚀 = 20."
    },
    {
        "problem": "🌟 + 🌟 = 🌈 + 4, 🌈 = 2 × 🌟 - 10",
        "solution": "I have two equations: 🌟 + 🌟 = 🌈 + 4 and 🌈 = 2 × 🌟 - 10. Substituting the second equation into the first: 🌟 + 🌟 = (2 × 🌟 - 10) + 4, so 2🌟 = 2🌟 - 6, 0 = -6, which is a contradiction. This system has no solution."
    }
]

formatted_data = []
for item in emoji_math_data:
    formatted_data.append({
        "input": f"Solve this emoji math problem: {item['problem']}",
        "output": item["solution"]
    })

dataset = Dataset.from_list(formatted_data)

dataset = dataset.train_test_split(test_size=0.1, seed=42)

model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

def process_function(examples):
    inputs = [f"### Instruction:\nSolve this emoji math problem: {ex}\n\n### Response:\n" for ex in examples["input"]]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    labels = tokenizer(
        examples["output"],
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    model_inputs["labels"] = labels["input_ids"].clone()

    return model_inputs

tokenized_datasets = {
    "train": Dataset.from_dict({
        "input": dataset["train"]["input"],
        "output": dataset["train"]["output"]
    }).map(process_function, batched=True),

    "test": Dataset.from_dict({
        "input": dataset["test"]["input"],
        "output": dataset["test"]["output"]
    }).map(process_function, batched=True)
}

class EmojiMathDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.input_ids = dataset["input_ids"]
        self.attention_mask = dataset["attention_mask"]
        self.labels = dataset["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }

train_dataset = EmojiMathDataset(tokenized_datasets["train"])
eval_dataset = EmojiMathDataset(tokenized_datasets["test"])

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./emoji-math-solver-results",
    num_train_epochs=5,  
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-4, 
    weight_decay=0.01,
    fp16=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collator,
)

trainer.train()

model.save_pretrained("./emoji-math-solver-model")
tokenizer.save_pretrained("./emoji-math-solver-model")

def solve_emoji_problem(problem, model, tokenizer):
    prompt = f"### Instruction:\nSolve this emoji math problem: {problem}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(prompt, "").strip()
    return response


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Solution: To solve this emoji math problem, we need to determine the value of the 🚗 emoji.

The equation is: 🚗 + 🚗 + 🚗 + 🚗 = 20

Since all the emojis are the same, we can simplify the equation by dividing both sides by 4:

🚗 = 20 / 4

🚗 = 5

Therefore, the value of the 🚗 emoji is 5.
The answer is $\boxed{5}$.

----------------------------

Problem: 🍎 × 🍎 = 16 → 🍎 = ?


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Solution: The answer is $\boxed{4}$.

Explanation: The emoji math problem 🍎 × 🍎 = 16 is asking for the value of 🍎 that satisfies the equation. Since the same emoji is used for both variables, we can assume that they are the same value. Therefore, we can rewrite the equation as 🍎² = 16. Taking the square root of both sides, we find that 🍎 = 4.

----------------------------

Problem: 🌮 + 🌮 + 🐱 = 17, 🐱 - 🌮 = 1 → 🌮 = ? and 🐱 = ?
Solution: To solve this emoji math problem, we need to determine the values of 🌮 (burrito) and 🐱 (cat) that satisfy the given equations.

Let's break down the information given:

1. 🌮 + 🌮 + 🐱 = 17
2. 🐱 - 🌮 = 1

From the second equation, we can express 🐱 in terms of 🌮:

🐱 = 1 + 🌮

Now, we can substitute this expression for 🐱 into the first equation:

🌮 + 🌮 + (1 + 🌮) = 17

Simplify the equation:

3🌮 + 1 = 17

Subtract 1 from both sides:

3🌮 = 16

Divide both sides by 3:

🌮 = 16/3 ≈ 5.33

Now that we have the value of 🌮, we can substitute it back into the expression f

KeyboardInterrupt: Interrupted by user

In [4]:
import random

def generate_emoji_rating():
    emojis = ["🧠", "🔢", "🧮", "📐", "🤔", "🎯", "💡", "✨"]
    math_iq = random.randint(130, 200)
    emoji_fluency = random.randint(80, 99)
    emoji_rating = "".join(random.choices(emojis, k=random.randint(3, 5)))

    print(f"Emoji Math IQ: {math_iq}")
    print(f"Emoji Fluency: {emoji_fluency}%")
    print(f"Emoji Rating: {emoji_rating}")


user_problem = input("\nEnter an emoji math problem: ")
solution = solve_emoji_problem(user_problem, model, tokenizer)
print(f"\nSolution: {solution}")

generate_emoji_rating()
print("Thank you for using the Emoji Math Solver!")


Enter an emoji math problem: 🌈 + 🌈 + 🌈 = 24


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.



Solution: 🌈 + 🌈 + 🌈 = 24
There are 8 eyes in total, so each eye represents the number 8.
Therefore, the answer is $\boxed{8}$.
Emoji Math IQ: 150
Emoji Fluency: 93%
Emoji Rating: 🧮🔢💡🧠
Thank you for using the Emoji Math Solver!


Streamlit Code For Frontend

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set page configuration
st.set_page_config(
    page_title="Emoji Math Solver",
    page_icon="🧮",
    layout="centered"
)

st.title("🧮 Emoji Math Solver")
st.markdown("Solve mathematical problems written with emojis! Enter your emoji math problem below.")

@st.cache_resource
def load_model():
    try:
        base_model_name = "deepseek-ai/deepseek-math-7b-instruct"
        tokenizer = AutoTokenizer.from_pretrained("./emoji-math-solver-model")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            torch_dtype=torch.float32,
            device_map="cpu"
        )
        model = PeftModel.from_pretrained(base_model, "./emoji-math-solver-model")
        return model, tokenizer
    except Exception as e:
        st.error(f"Error loading model: {str(e)}")
        return None, None

def solve_emoji_problem(problem, model, tokenizer):
    prompt = f"### Instruction:\nSolve this emoji math problem: {problem}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.replace(prompt, "").strip()

example_problems = [
    "🍎 + 🍎 + 🍎 = 12",
    "🚗 + 🚗 + 🚗 + 🚗 = 20",
    "🍎 × 🍎 = 16",
    "🌮 + 🌮 + 🐱 = 17, 🐱 - 🌮 = 1"
]

with st.sidebar:
    st.header("About")
    st.markdown("Solve math problems written with emojis!")

st.header("Enter Your Problem")
use_example = st.checkbox("Use an example problem")

if use_example:
    problem_input = st.selectbox("Select an example problem:", example_problems)
else:
    problem_input = st.text_input("Enter your emoji math problem:", placeholder="e.g., 🍎 + 🍎 + 🍎 = 12")

if st.button("Solve Problem"):
    if problem_input:
        with st.spinner("Solving the problem..."):
            model, tokenizer = load_model()
            if model and tokenizer:
                solution = solve_emoji_problem(problem_input, model, tokenizer)
                st.header("Solution")
                st.markdown(f"**Problem:** {problem_input}")
                st.markdown(f"**Solution:** {solution}")
                st.balloons()
            else:
                st.error("Failed to load the model. Please check your model files.")
    else:
        st.warning("Please enter a problem to solve.")

st.markdown("---")
st.markdown("Created with ❤️ and 🧮")


Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.102.77:8501

